In [36]:
import pyspark
from delta import *
from delta.tables import *
from pyspark.sql.functions import *

In [2]:
warehouse_location = 'hdfs://hdfs:9000/hive/warehouse'

In [3]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp").master('spark://master:7077') \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir", warehouse_location)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ee2ce7dc-bfca-4ea3-8f5d-179cf96ed6a5;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 695ms :: artifacts dl 22ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runtime;3.5.2 fro

In [4]:
spark

In [7]:
!$HADOOP_HOME/bin/hdfs dfs -rm -r hdfs://hdfs:9000/hive/warehouse/people10m

Deleted hdfs://hdfs:9000/hive/warehouse/people10m


In [8]:
sql = """
CREATE TABLE IF NOT EXISTS default.people10m (
  id INT,
  firstName STRING,
  middleName STRING,
  lastName STRING,
  gender STRING,
  birthDate TIMESTAMP,
  ssn STRING,
  salary INT
) USING DELTA
"""
spark.sql(sql)

DataFrame[]

In [9]:
sql = """
INSERT INTO default.people10m
values (1, 'husein', 'bin', 'zolkepli', 'm', '2021-01-01 00:00:01', '1', 100)
"""
spark.sql(sql)

DataFrame[]

In [13]:
df_sql = spark.sql('select * from default.people10m')
df_sql.show()

+---+---------+----------+--------+------+-------------------+---+------+
| id|firstName|middleName|lastName|gender|          birthDate|ssn|salary|
+---+---------+----------+--------+------+-------------------+---+------+
|  1|   husein|       bin|zolkepli|     m|2021-01-01 00:00:01|  1|   100|
+---+---------+----------+--------+------+-------------------+---+------+



In [11]:
DeltaTable.createIfNotExists(spark) \
  .tableName("default.people10m") \
  .addColumn("id", "INT") \
  .addColumn("firstName", "STRING") \
  .addColumn("middleName", "STRING") \
  .addColumn("lastName", "STRING", comment = "surname") \
  .addColumn("gender", "STRING") \
  .addColumn("birthDate", "TIMESTAMP") \
  .addColumn("ssn", "STRING") \
  .addColumn("salary", "INT") \
  .execute()

In [12]:
DeltaTable.createOrReplace(spark) \
  .addColumn("id", "INT") \
  .addColumn("firstName", "STRING") \
  .addColumn("middleName", "STRING") \
  .addColumn("lastName", "STRING") \
  .addColumn("gender", "STRING") \
  .addColumn("birthDate", "TIMESTAMP") \
  .addColumn("ssn", "STRING") \
  .addColumn("salary", "INT") \
  .location(f"{warehouse_location}/people10m") \
  .execute()

In [23]:
from datetime import datetime

data = [(2, 'hasan', 'bin', 'zolkepli', 'm', 
         datetime.strptime('2000-01-01 00:00:01', '%Y-%m-%d %H:%M:%S'), '2', 1000),
        (3, 'husna', 'binti', 'zolkepli', 'f', 
         datetime.strptime('1996-01-01 00:00:01', '%Y-%m-%d %H:%M:%S'), '3', 500),
        (4, 'ayu', 'binti', 'dzul', 'f', 
         datetime.strptime('1990-01-01 00:00:01', '%Y-%m-%d %H:%M:%S'), '4', 100),]
  
dataframe = spark.createDataFrame(data, df_sql.schema)
dataframe.show()

+---+---------+----------+--------+------+-------------------+---+------+
| id|firstName|middleName|lastName|gender|          birthDate|ssn|salary|
+---+---------+----------+--------+------+-------------------+---+------+
|  2|    hasan|       bin|zolkepli|     m|2000-01-01 00:00:01|  2|  1000|
|  3|    husna|     binti|zolkepli|     f|1996-01-01 00:00:01|  3|   500|
|  4|      ayu|     binti|    dzul|     f|1990-01-01 00:00:01|  4|   100|
+---+---------+----------+--------+------+-------------------+---+------+



In [24]:
dataframe.write.format('delta').mode('append').saveAsTable("default.people10m")

In [25]:
df_sql = spark.sql('select * from default.people10m')
df_sql.show()

+---+---------+----------+--------+------+-------------------+---+------+
| id|firstName|middleName|lastName|gender|          birthDate|ssn|salary|
+---+---------+----------+--------+------+-------------------+---+------+
|  1|   husein|       bin|zolkepli|     m|2021-01-01 00:00:01|  1|   100|
|  2|    hasan|       bin|zolkepli|     m|2000-01-01 00:00:01|  2|  1000|
|  3|    husna|     binti|zolkepli|     f|1996-01-01 00:00:01|  3|   500|
|  4|      ayu|     binti|    dzul|     f|1990-01-01 00:00:01|  4|   100|
+---+---------+----------+--------+------+-------------------+---+------+



In [34]:
deltaTable = DeltaTable.forName(spark, 'default.people10m')

In [39]:
deltaTable.delete("birthDate < '1955-01-01'")
deltaTable.delete(col('birthDate') < '1995-01-011')

In [40]:
df_sql = spark.sql('select * from default.people10m')
df_sql.show()

+---+---------+----------+--------+------+-------------------+---+------+
| id|firstName|middleName|lastName|gender|          birthDate|ssn|salary|
+---+---------+----------+--------+------+-------------------+---+------+
|  3|    husna|     binti|zolkepli|     f|1996-01-01 00:00:01|  3|   500|
|  1|   husein|       bin|zolkepli|     m|2021-01-01 00:00:01|  1|   100|
|  2|    hasan|       bin|zolkepli|     m|2000-01-01 00:00:01|  2|  1000|
+---+---------+----------+--------+------+-------------------+---+------+



In [41]:
deltaTable.update(
  condition = "gender = 'f'",
  set = { "gender": "'Female'" }
)

# Declare the predicate by using Spark SQL functions.
deltaTable.update(
  condition = col('gender') == 'm',
  set = { 'gender': lit('Male') }
)

In [42]:
df_sql = spark.sql('select * from default.people10m')
df_sql.show()

+---+---------+----------+--------+------+-------------------+---+------+
| id|firstName|middleName|lastName|gender|          birthDate|ssn|salary|
+---+---------+----------+--------+------+-------------------+---+------+
|  3|    husna|     binti|zolkepli|Female|1996-01-01 00:00:01|  3|   500|
|  1|   husein|       bin|zolkepli|  Male|2021-01-01 00:00:01|  1|   100|
|  2|    hasan|       bin|zolkepli|  Male|2000-01-01 00:00:01|  2|  1000|
+---+---------+----------+--------+------+-------------------+---+------+



In [43]:
new_data = [(2, 'hasan', 'bin', 'zolkepli', 'm', 
         datetime.strptime('2000-01-01 00:00:01', '%Y-%m-%d %H:%M:%S'), '2', 1000),
        (3, 'husna', 'binti', 'zolkepli', 'f', 
         datetime.strptime('1996-01-01 00:00:01', '%Y-%m-%d %H:%M:%S'), '3', 500),
        (4, 'ayu', 'binti', 'dzul', 'f', 
         datetime.strptime('1990-01-01 00:00:01', '%Y-%m-%d %H:%M:%S'), '4', 100),
        (5, 'ayu-v2', 'binti', 'dzul', 'f', 
         datetime.strptime('1990-01-01 00:00:01', '%Y-%m-%d %H:%M:%S'), '5', 100)]
  
dataframe = spark.createDataFrame(new_data, df_sql.schema)
dataframe.show()

+---+---------+----------+--------+------+-------------------+---+------+
| id|firstName|middleName|lastName|gender|          birthDate|ssn|salary|
+---+---------+----------+--------+------+-------------------+---+------+
|  2|    hasan|       bin|zolkepli|     m|2000-01-01 00:00:01|  2|  1000|
|  3|    husna|     binti|zolkepli|     f|1996-01-01 00:00:01|  3|   500|
|  4|      ayu|     binti|    dzul|     f|1990-01-01 00:00:01|  4|   100|
|  5|   ayu-v2|     binti|    dzul|     f|1990-01-01 00:00:01|  5|   100|
+---+---------+----------+--------+------+-------------------+---+------+



In [44]:
dataframe.write.format('delta').mode('append').saveAsTable("default.people10m_update")

In [45]:
deltaTablePeople = DeltaTable.forName(spark, 'default.people10m')
deltaTablePeopleUpdates = DeltaTable.forName(spark, 'default.people10m_update')

In [48]:
dfUpdates = deltaTablePeopleUpdates.toDF()
dfUpdates

DataFrame[id: int, firstName: string, middleName: string, lastName: string, gender: string, birthDate: timestamp, ssn: string, salary: int]

In [49]:
deltaTablePeople.alias('people') \
  .merge(
    dfUpdates.alias('updates'),
    'people.id = updates.id'
  ) \
  .whenMatchedUpdate(set =
    {
      "id": "updates.id",
      "firstName": "updates.firstName",
      "middleName": "updates.middleName",
      "lastName": "updates.lastName",
      "gender": "updates.gender",
      "birthDate": "updates.birthDate",
      "ssn": "updates.ssn",
      "salary": "updates.salary"
    }
  ) \
  .whenNotMatchedInsert(values =
    {
      "id": "updates.id",
      "firstName": "updates.firstName",
      "middleName": "updates.middleName",
      "lastName": "updates.lastName",
      "gender": "updates.gender",
      "birthDate": "updates.birthDate",
      "ssn": "updates.ssn",
      "salary": "updates.salary"
    }
  ) \
  .execute()

22/01/21 03:59:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [50]:
df_sql = spark.sql('select * from default.people10m')
df_sql.show()

+---+---------+----------+--------+------+-------------------+---+------+
| id|firstName|middleName|lastName|gender|          birthDate|ssn|salary|
+---+---------+----------+--------+------+-------------------+---+------+
|  1|   husein|       bin|zolkepli|  Male|2021-01-01 00:00:01|  1|   100|
|  3|    husna|     binti|zolkepli|     f|1996-01-01 00:00:01|  3|   500|
|  2|    hasan|       bin|zolkepli|     m|2000-01-01 00:00:01|  2|  1000|
|  5|   ayu-v2|     binti|    dzul|     f|1990-01-01 00:00:01|  5|   100|
|  4|      ayu|     binti|    dzul|     f|1990-01-01 00:00:01|  4|   100|
+---+---------+----------+--------+------+-------------------+---+------+

